Enter the timestack image name to be processed and then run the cell (shift+enter). Also enter the tf_model_version you would like to use. These files must be in the folder of this notebook.

In [ ]:
timestack_name = "example_timestack.png" # this example timestack was created by Associate Professor Hannah Power
tf_model_version = "michaelThompson_imgSize_64"
IMG_SIZE = 64 # the size of image that the selected tensorflow model uses
IMG_SIZE_WID = 64 # sometimes this is different - if the image is taken as rectangular

Run the next cell to open a GUI for cropping the timestack before automated tracing. By scrolling through the entire timestack, find the minimum rundown point and maximum runup point. Also set the green slider to the height of 5 medium sized swash event peaks. The swash front tracing model was optimised to work on images including approximately 5 swash peaks, focused on the runup region of the swash zone.

In [ ]:
# import libraries
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
import csv
import tensorflow as tf
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
import random
from scipy.stats import norm

title_window = "set the height of 5 peaks, the max runup point, the min rundown point"

# get the timestack image
timestack = cv2.imread(timestack_name)
height, width, _ = timestack.shape
padding = width

# initial slider values
min_uprush = 306 # suitable value for 20101109084810_06_25ppm.png
max_uprush = 896 # suitable value for 20101109084810_06_25ppm.png
five_peaks_height = 296 # suitable value for 20101109084810_06_25ppm.png
scroll_image = 0

"""%%%%%%%%%%%%%%%%%%%% STEP 1: FIND HEIGHT OF 5 PEAKS, MAX RUNUP, MIN RUNDOWN %%%%%%%%%%%%%"""
timestack = cv2.imread(timestack_name) ################ NOT VERY EFFICIENT
timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

# show a horizontal line indicating the row to be marked
cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0], thickness=2)
cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
         thickness=2)
cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
         thickness=2)
timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
cv2.imshow(title_window, timestack_snapshot)

# add a method to label each image with an x-coordinate (using a slider) and whether it is min, max or neither (slider)
def on_trackbar_min_uprush(val):
    global min_uprush

    min_uprush = val
    timestack = cv2.imread(timestack_name)  ################ NOT VERY EFFICIENT
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show a horizontal line indicating the row to be marked
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0], thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

# add a method to label each image with an x-coordinate (using a slider) and whether it is min, max or neither (slider)
def on_trackbar_max_uprush(val):
    global max_uprush

    max_uprush = val
    timestack = cv2.imread(timestack_name)  ################ NOT VERY EFFICIENT
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show a horizontal line indicating the row to be marked
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0],
             thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

# add a method to label each image with an x-coordinate (using a slider) and whether it is min, max or neither (slider)
def on_trackbar_five_peaks_height(val):
    global five_peaks_height

    five_peaks_height = val
    timestack = cv2.imread(timestack_name)  ################ NOT VERY EFFICIENT
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show a horizontal line indicating the row to be marked
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0],
             thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

# add a method to label each image with an x-coordinate (using a slider) and whether it is min, max or neither (slider)
def on_trackbar_scroll_image(val):
    global scroll_image

    scroll_image = val
    timestack = cv2.imread(timestack_name)  ################ NOT VERY EFFICIENT
    timestack_snapshot = timestack[scroll_image:scroll_image+padding, 0:]

    # show a horizontal line indicating the row to be marked
    cv2.line(timestack_snapshot, (0, int(five_peaks_height)), (padding, int(five_peaks_height)), [0, 255, 0],
             thickness=2)
    cv2.line(timestack_snapshot, (int(min_uprush), 0), (int(min_uprush), int(padding)), [0, 0, 255],
             thickness=2)
    cv2.line(timestack_snapshot, (int(max_uprush), 0), (int(max_uprush), int(padding)), [255, 0, 0],
             thickness=2)
    timestack_snapshot = cv2.resize(timestack_snapshot, (720, 540))
    cv2.imshow(title_window, timestack_snapshot)

# setup trackbar
trackbar_min_uprush = "Rd_min"
trackbar_max_uprush = "Ru_max"
trackbar_five_peaks_height = "L_5p"
trackbar_scroll_image = "Scroll"

cv2.createTrackbar(trackbar_min_uprush, title_window, min_uprush, padding, on_trackbar_min_uprush)
cv2.createTrackbar(trackbar_max_uprush, title_window, max_uprush, padding, on_trackbar_max_uprush)
cv2.createTrackbar(trackbar_five_peaks_height, title_window, five_peaks_height, padding, on_trackbar_five_peaks_height)
if height - padding > 0:
    cv2.createTrackbar(trackbar_scroll_image, title_window, scroll_image, height - padding, on_trackbar_scroll_image)
cv2.waitKey()

print("done")

Run model on full timestack

In [ ]:
timestack = cv2.imread(timestack_name)
image_range = max_uprush - min_uprush
timestack_focus = timestack[:, min_uprush:max_uprush]
timestack_annotated = timestack.copy()

height_of_two_peaks = int(five_peaks_height*(2/5))
# for every row, find the shoreline location (plot as blue dots). If it is a minimum point plot as red Dot, If maximum plot as blue Dot
padding = height_of_two_peaks
height, width, _ = timestack.shape
width = max_uprush - min_uprush
timestack_count = 0

IMG_SIZE_WID = int(IMG_SIZE) # can change this if needed
tf_images = []
# this way of reading/saving models worked for an older version of tensorflow 2.x
tf_model_xc = tf.keras.models.load_model(tf_model_version + ".model")

vertical_coordinates = []
horizontal_coordinates = []
for i in range(0, int(height - padding), 1):
    if i % int(int(height - padding)/20) == 0:
        print("processing at " + str(round(100*i/int(height - padding),0)) + "%")
    
    horizontal_coordinate = i + int(padding/2)

    # get the small section of the image for the horizontal position i
    timestack_snapshot = timestack_focus[i:i + padding, :]

    # classify the timestack snapshot through the convolutional neural network for X_coordinate
    #tf_images = cv2.cvtColor(timestack_snapshot, cv2.COLOR_BGR2GRAY)   # should have this for gray
    tf_images = cv2.resize(timestack_snapshot, (IMG_SIZE_WID, IMG_SIZE))
    tf_images_i = np.array(tf_images)
    tf_images_i = tf.keras.utils.normalize(tf_images_i, axis=1)
    tf_images_i = tf_images_i.reshape(-1, IMG_SIZE, IMG_SIZE_WID, 3)   #should be changed to 1 for gray
    prediction = tf_model_xc.predict([tf_images_i])

    vertical_coordinate = int((prediction[0])*width) + min_uprush

    cv2.circle(timestack_annotated, (vertical_coordinate, horizontal_coordinate), radius=1, color=[255, 0, 0])
    vertical_coordinates.append(vertical_coordinate)
    horizontal_coordinates.append(horizontal_coordinate)

    timestack_count = timestack_count + 1

print("red is found by model")
plt.plot(vertical_coordinates, horizontal_coordinates, color = 'r')
plt.gca().invert_yaxis() # invert axis so it is like the timestack image coordinates
plt.show()

# save the image
print("saving image")
cv2.imwrite("annotated_runup_timestack_" + timestack_name[:-4] + "_" + tf_model_version + ".png", timestack_annotated)

# save the data
print("Saving Data to csv file")

# create csv file
output_data_name = "runup_data_" + timestack_name[:-4] + "_" + tf_model_version + ".csv"
with open(output_data_name, 'w') as writeFile:
    writer = csv.writer(writeFile)
    writer.writerows([["timestack shoreline coordinates"]])

with open(output_data_name, 'a', newline='') as csvFile:
    writer = csv.writer(csvFile)

    time_row = list(["time (horizontal coordinate)"])
    time_row.extend(horizontal_coordinates)
    x_row = list(["x (vertical coordinate)"])
    x_row.extend(vertical_coordinates)
    writer.writerow(time_row)
    writer.writerow(x_row)

csvFile.close()

print("done")